# Notebook 4 - results plotting

## Intro

This Jupyter notebook contains parts of modeling behind the publication: 
> Krych, K. & Pettersen, JB. (2024). Long-term lifetime trends of large appliances since the introduction in Norwegian households. Journal of Industrial Ecology. 

Here, we combine the results from the other Jupyter Notebooks in order to summarize and plot them. 

## Imports and simulation setup

In [ ]:
import os
import sys
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import pickle
import scipy.stats
import SALib.analyze.sobol

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), os.pardir, 'odym/odym/modules')))
import ODYM_Classes as msc # import the ODYM class file

In [ ]:
durables = ['fridges' ,'freezers','washing machines', 'tumble dryers', 'dishwashers', 'ovens']
TimeStart = 1940
TimeEnd = 2022
MyYears = list(range(TimeStart,TimeEnd+1))
excel = os.path.abspath(os.path.join(os.getcwd(), 'data.xlsx'))
export_data_to_xlsx = True
export_figs_to_pdf = True

In [ ]:
df_i_data = pd.read_excel(excel, sheet_name='I_data')
df_i_ip = pd.read_excel(excel, sheet_name='I')
df_popd = pd.read_excel(excel, sheet_name='POpD_data')
df_lt = pd.read_excel(excel, sheet_name='LT_data')
unique_i = list(df_i_data["data type"].unique())
unique_popd = list(df_popd["unit"].unique())
palette_data_types = dict(zip(unique_i+unique_popd, sns.color_palette("colorblind", n_colors=len(unique_i+unique_popd))))

In [ ]:
with open("3a_analysis_constant.dat", "rb") as f:
    [Dyn_MFA_System1, problem1, param_values1, no_of_runs1] = pickle.load(f)

In [ ]:
with open("3b_analysis_variable.dat", "rb") as f:
    [Dyn_MFA_System2, problem2, param_values2, no_of_runs2] = pickle.load(f)

In [ ]:
with open("2_preliminary_analysis_global.dat", "rb") as f:
    [Dyn_MFA_System3, problem3, param_values3, no_of_runs3] = pickle.load(f)

In [ ]:
def get_array_with_nans(array, indices, no_of_runs):
    max_no_of_runs = max(no_of_runs.values())
    index_d = indices.index('d')
    index_r = indices.index('r')
    for d,durable in enumerate(durables):
        index = [slice(None)]*len(indices)
        index[index_d] = d
        index[index_r] = np.r_[no_of_runs[durable]:max_no_of_runs]
        array[tuple(index)] = np.nan
    return array

In [ ]:
def calculate_mean_and_percentiles(array,index_r):
    avg = np.nanmean(array, axis=index_r).flatten(order='F')
    perc_0th = np.nanpercentile(array, 0, axis=index_r).flatten(order='F')
    perc_5th = np.nanpercentile(array, 5, axis=index_r).flatten(order='F')
    perc_25th = np.nanpercentile(array, 25, axis=index_r).flatten(order='F')
    perc_50th = np.nanpercentile(array, 50, axis=index_r).flatten(order='F')
    perc_75th = np.nanpercentile(array, 75, axis=index_r).flatten(order='F')
    perc_95th = np.nanpercentile(array, 95, axis=index_r).flatten(order='F')
    perc_100th = np.nanpercentile(array, 100, axis=index_r).flatten(order='F')
    data = np.concatenate(([avg],[perc_0th],[perc_5th],[perc_25th],[perc_50th],[perc_75th],[perc_95th],[perc_100th]), axis=0)
    col_names = ['average', '0th percentile', '5th percentile', '25th percentile', '50th percentile', '75th percentile', '95th percentile', '100th percentile']
    return data, col_names

## Main manuscript

### Figure 1 - inflows & ownership

In [ ]:
def calculate_xlim(df_popd_d, df_i_d, uncertainty=True, u=0.05):
    popd_mean = df_popd_d[df_popd_d['time'].isin(range(1985,1996))]['value'].mean()
    popd_max = df_popd_d['value'].max()
    i_mean = df_i_d[df_i_d['time'].isin(range(1985,1996))]['value'].mean()
    if uncertainty:
        i_max = df_i_d['value'].max()*1.02*(1+u*1.96)
    else:
        i_max = df_i_d['value'].max()*1.05
    popd_share = popd_mean/popd_max
    i_share = i_mean/i_max
    if i_share<popd_share:
        i_ylim = i_max
        popd_ylim = popd_max*popd_share/i_share
    else:
        i_ylim = i_max*i_share/popd_share
        popd_ylim = popd_max
    return i_ylim, popd_ylim

In [ ]:
columns = 2
rows = 3
fig = plt.figure(figsize=(5*columns, 3*rows), constrained_layout=True)
fig.set_dpi(800) #min 600 for publication
gs = fig.add_gridspec(rows, columns,hspace=0.1,wspace=0.05)
row = 0
col = 0
u = 0.05
for d,durable in enumerate(durables):
    if durable == durables[0]:
        labels = ['(right axis) Ownership, data', '(left axis) Sales/year, data', '(left axis) Sales/year, averaged and interpolated', '(left axis) Sales/year, assumed 95% uncertainty interval']
    else:
        labels = ['_nolegend_']*4
    df_popd_d = df_popd[(df_popd['durable'] == durable) | (df_popd['durable'] == durable+' [excluded]')]
    df_i_d = df_i_data[df_i_data['durable'] == durable]
    df_i_ip_d = df_i_ip[df_i_ip['durable'] == durable]
    i_ylim, popd_ylim = calculate_xlim(df_popd_d, df_i_d)
    df_popd_d = df_popd_d[df_popd_d['source'] != 'assumption']
    ax1 = fig.add_subplot(gs[row, col])
    ax2 = ax1.twinx()
    # sns.scatterplot(data=df_popd_d, x='time', y='value', hue='data type', s=50, palette=palette_data_types, ax=ax1)
    sns.scatterplot(data=df_i_d, x='time', y='value', s=30, ax=ax1, marker="o", color='C2', alpha=0.8, label=labels[1])
    sns.scatterplot(data=df_popd_d, x='time', y='value', s=20, ax=ax2, marker="D", color=".2", alpha=1, label=labels[0])
    sns.lineplot(data=df_i_ip_d, x='time', y='value', ax=ax1, color='C2', alpha=0.8, label=labels[2])
    ax1.fill_between(MyYears, df_i_ip_d['value']*(1-u*1.96), df_i_ip_d['value']*(1+u*1.96), color='gray', alpha=0.5, label=labels[3],zorder=0)
    # ax2.axhline(y=0.25,color="grey", linestyle="--")
    ax2.axhline(y=0.5,color="grey", linestyle="--", alpha=0.5)
    # ax2.axhline(y=0.75,color="grey", linestyle="--")
    ax2.axhline(y=1,color="grey", linestyle="--", alpha=0.5)
    """if durable in ['fridges', 'washing machines', 'ovens']:
        ax1.axvline(x=1960,color="grey", linestyle="--", alpha=0.5)
    ax1.axvline(x=1970,color="grey", linestyle="--", alpha=0.5)
    ax1.axvline(x=1980,color="grey", linestyle="--", alpha=0.5)
    ax1.axvline(x=1990,color="grey", linestyle="--", alpha=0.5)
    ax1.axvline(x=2000,color="grey", linestyle="--", alpha=0.5)
    ax1.axvline(x=2010,color="grey", linestyle="--", alpha=0.5)"""
    ax1.legend('')
    ax2.legend('')
    ax1.get_legend().remove()
    ax2.get_legend().remove()
    ax1.set_xlabel('Year')
    ax1.set_ylabel('Sales/year')
    ax2.set_ylabel('Ownership')
    ax1.set_xlim(TimeStart-4, TimeEnd+4)
    ax1.set_ylim(0,i_ylim)
    ax2.set_ylim(0,popd_ylim)
    ax1.set_title(durable.capitalize())
    if col == columns-1:
        col = 0
        row+= 1
    else:
        col += 1
lines_labels = [ax.get_legend_handles_labels() for ax in fig.axes]
lines, labels = [sum(lol, []) for lol in zip(*lines_labels)]
fig.legend(lines, labels, bbox_to_anchor=(0.5,0), loc="upper center", fontsize=12)
plt.show()
if export_figs_to_pdf:
    fig.savefig('outputs/Fig1.pdf', format='pdf', bbox_inches = "tight")

### Figure 2 - ownership model vs data

In [ ]:
x = Dyn_MFA_System1.IndexTable.Classification['Time'].Items
columns = 2
rows = 3
fig = plt.figure(figsize=(6*columns, 3.5*rows), constrained_layout=True)
fig.set_dpi(800) #min 600 for publication
gs = fig.add_gridspec(rows, columns,hspace=0.1)
row = 0
col = 0
for d,durable in enumerate(Dyn_MFA_System1.IndexTable['Classification']['Durable'].Items):
    df_popd_d = df_popd[df_popd['durable'] == durable]
    df_popd_d_excluded = df_popd[df_popd['durable'] == durable+' [excluded]']
    ax = fig.add_subplot(gs[row, col])
    for r in range(no_of_runs1[durable]):
        if durable == durables[0] and r == 0:
            labels = ['Constant lifetime model results']
        else:
            labels = ['_nolegend_']
        ax.plot(x, Dyn_MFA_System1.ExtraDict['POpD'].Values[:,d,r], color='C0', alpha=0.05, label=labels[0])
    for r in range(no_of_runs2[durable]):
        if durable == durables[0] and r == 0:
            labels = ['Variable lifetime model results']
        else:
            labels = ['_nolegend_']
        ax.plot(x, Dyn_MFA_System2.ExtraDict['POpD'].Values[:,d,r], color='C1', alpha=0.05, label=labels[0])
    if durable == durables[0]:
        labels = ['Data', 'Assumption']
    else:
        labels = ['_nolegend_']*2
    sns.scatterplot(data=df_popd_d[df_popd_d['source'] != 'assumption'], x='time', y='value', s=40, marker="D", color="black", label=labels[0],zorder=r+1)
    sns.scatterplot(data=df_popd_d[df_popd_d['source'] == 'assumption'], x='time', y='value', s=40, marker="D", color="gray", label=labels[1],zorder=r+1)
    sns.scatterplot(data=df_popd_d_excluded, x='time', y='value', s=40, marker="D", color="red", label='Excluded data',zorder=r+1)
    ax.set_xlabel('Year')
    ax.set_ylabel('Ownership')
    ax.set_title(durable.capitalize())
    ax.set_xlim(1940-4,2022+4)
    ax.legend('')
    ax.get_legend().remove()
    if col == columns-1:
        col = 0
        row+= 1
    else:
        col += 1
lines_labels = [ax.get_legend_handles_labels() for ax in fig.axes]
lines, labels = [sum(lol, []) for lol in zip(*lines_labels)]
leg = fig.legend(lines, labels, bbox_to_anchor=(0.5,0), loc="upper center", fontsize=12)
for l in leg.get_lines(): # full answer: https://stackoverflow.com/questions/35200094/change-size-alpha-of-markers-in-the-legend-box
    l.set_alpha(1)
    # l.set_linewidth(5)
plt.show()
if export_figs_to_pdf:
    fig.savefig('outputs/Fig2.pdf', format='pdf', bbox_inches = "tight")

In [ ]:
if export_data_to_xlsx:
    parameter = Dyn_MFA_System1.ExtraDict['POpD']
    array = get_array_with_nans(parameter.Values, parameter.Indices, no_of_runs1)
    data, col_names = calculate_mean_and_percentiles(array,parameter.Indices.index('r'))
    df = pd.DataFrame(data=data.T, index=pd.MultiIndex.from_product([durables, MyYears], names=['durable','time']), columns=col_names)
    df.to_excel('outputs/Fig2_constant.xlsx')

In [ ]:
if export_data_to_xlsx:
    parameter = Dyn_MFA_System2.ExtraDict['POpD']
    array = get_array_with_nans(parameter.Values, parameter.Indices, no_of_runs2)
    data, col_names = calculate_mean_and_percentiles(array,parameter.Indices.index('r'))
    df = pd.DataFrame(data=data.T, index=pd.MultiIndex.from_product([durables, MyYears], names=['durable','time']), columns=col_names)
    df.to_excel('outputs/Fig2_variable.xlsx')

### Figure 3 - outflows

In [ ]:
df_w = pd.read_excel(excel, sheet_name='W')
df_w['durable'] = df_w['durable'].astype("category")
df_w['durable'] = df_w['durable'].cat.set_categories(durables)
df_w = df_w.sort_values(by=['durable'])
weight = list(df_w['value'])

In [ ]:
app_groups = {'cooling appliances': ['fridges', 'freezers'],
            'other big appliances': ['washing machines', 'tumble dryers', 'dishwashers', 'ovens']}
outflows_constant = np.zeros((len(MyYears),2,min(no_of_runs1.values())))
outflows_logistic = np.zeros((len(MyYears),2,min(no_of_runs2.values())))
for g, cat in enumerate(app_groups.keys()):
    durables_cat = [item for item in app_groups[cat] if item in durables]
    d_cat= [Dyn_MFA_System1.IndexTable['Classification']['Durable'].Items.index(durable) for durable in durables_cat]
    weight_cat = [weight[d] for d in d_cat]
    for r in range(min(no_of_runs1.values())):
        y = 0
        for i, d in enumerate(d_cat):
            weight_d = weight_cat[i]
            y += Dyn_MFA_System1.ExtraDict['F_10'].Values[:,d,r]*weight_d/1000
        outflows_constant[:,g,r] = y
    for r in range(min(no_of_runs2.values())):
        y = 0
        for i, d in enumerate(d_cat):
            weight_d = weight_cat[i]
            y += Dyn_MFA_System2.ExtraDict['F_10'].Values[:,d,r]*weight_d/1000
        outflows_logistic[:,g,r] = y

In [ ]:
df_o = pd.read_excel(excel, sheet_name='O_data')
x = Dyn_MFA_System1.IndexTable['Classification']['Time'].Items
fig = plt.figure(figsize=(10, 3), constrained_layout=True)
gs = fig.add_gridspec(1, 2,wspace=0.05)
row = 0
col = 0
for g,cat in enumerate(app_groups.keys()):
    ax = fig.add_subplot(gs[row, col])
    df_o_d = df_o[df_o['durable category'] == cat]
    y_val = df_o_d['value']
    x_val = df_o_d['time']
    fig.set_dpi(800) #min 600 for publication
    for r in range(min(no_of_runs1.values())):
        if r == 0 and cat == 'cooling appliances':
            labels = ['Constant lifetime model results']
        else:
            labels = ['_nolegend_']
        sns.lineplot(ax=ax, x=MyYears,y=outflows_constant[:,g,r], color='C0', alpha=0.05, label=labels[0],legend=False)
    for r in range(min(no_of_runs2.values())):
        if r == 0 and cat == 'cooling appliances':
            labels = ['Variable lifetime model results']
        else:
            labels = ['_nolegend_']
        sns.lineplot(ax=ax, x=MyYears,y=outflows_logistic[:,g,r], color='C1', alpha=0.05, label=labels[0],legend=False)
    if  cat == 'cooling appliances':
        labels = ['Data']
    else:
        labels = ['_nolegend_']
    sns.scatterplot(ax=ax, x=x_val, y=y_val, label=labels[0],zorder=r+1, color='black',legend=False)
    ax.set_ylabel(f'Outflows (tons)')
    ax.set_xlabel('Year')
    ax.set_title(cat.capitalize())
    col += 1
lines_labels = [ax.get_legend_handles_labels() for ax in fig.axes]
lines, labels = [sum(lol, []) for lol in zip(*lines_labels)]
leg = fig.legend(lines, labels, bbox_to_anchor=(0.5,0), loc="upper center", fontsize=12)
for l in leg.get_lines(): # full answer: https://stackoverflow.com/questions/35200094/change-size-alpha-of-markers-in-the-legend-box
    l.set_alpha(1)
plt.show()
if export_figs_to_pdf:
    fig.savefig('outputs/Fig3.pdf', format='pdf', bbox_inches = "tight")

In [ ]:
if export_data_to_xlsx:
    data, col_names = calculate_mean_and_percentiles(outflows_constant,2)
    df = pd.DataFrame(data=data.T, index=pd.MultiIndex.from_product([app_groups.keys(), MyYears], names=['category','time']), columns=col_names)
    df.to_excel('outputs/Fig3_constant.xlsx')

In [ ]:
if export_data_to_xlsx:
    data, col_names = calculate_mean_and_percentiles(outflows_logistic,2)
    df = pd.DataFrame(data=data.T, index=pd.MultiIndex.from_product([app_groups.keys(), MyYears], names=['category','time']), columns=col_names)
    df.to_excel('outputs/Fig3_variable.xlsx')

### Figure 4 - lifetimes

In [ ]:
x = Dyn_MFA_System1.IndexTable.Classification['Time'].Items
columns = 2
rows = 3
fig = plt.figure(figsize=(6*columns, 3.5*rows))
fig.set_dpi(800) #min 600 for publication
gs = fig.add_gridspec(rows, columns*9,hspace=0.4,wspace=0.1)
row = 0
col = 0
for d,durable in enumerate(Dyn_MFA_System1.IndexTable['Classification']['Durable'].Items):
    ax = fig.add_subplot(gs[row, col*9])
    sns.kdeplot(y=Dyn_MFA_System1.ExtraDict['mu'].Values[0,d,:no_of_runs1[durable]], color='C0', bw_adjust=1.5, alpha=1)
    sns.kdeplot(y=Dyn_MFA_System2.ExtraDict['mu'].Values[0,d,:no_of_runs2[durable]], color='C1', bw_adjust=1.5, alpha=1)
    ax.set_ylim(0, 38)
    ax.set_ylabel('Mean lifetime (years)')
    ax.invert_xaxis()
    ax.set_xlabel('')
    # ax.set_xlim(0, 1.1)
    ax.set_xticks([],labels=None)
    ax.axhline(y=10,color="grey", linestyle="--", alpha=0.5)
    ax.axhline(y=20,color="grey", linestyle="--", alpha=0.5)
    ax.axhline(y=30,color="grey", linestyle="--", alpha=0.5)

    ax = fig.add_subplot(gs[row, col*9+7])
    sns.kdeplot(y=Dyn_MFA_System1.ExtraDict['mu'].Values[-1,d,:no_of_runs1[durable]], color='C0', bw_adjust=1.5, alpha=1)
    sns.kdeplot(y=Dyn_MFA_System2.ExtraDict['mu'].Values[-1,d,:no_of_runs2[durable]], color='C1', bw_adjust=1.5, alpha=1)
    ax.set_ylim(0, 38)
    ax.set_ylabel('')
    ax.set_yticks([],labels=None)
    ax.set_xlabel('')
    # ax.set_xlim(0, 1.1)
    ax.set_xticks([],labels=None)
    ax.axhline(y=10,color="grey", linestyle="--", alpha=0.5)
    ax.axhline(y=20,color="grey", linestyle="--", alpha=0.5)
    ax.axhline(y=30,color="grey", linestyle="--", alpha=0.5)

    ax = fig.add_subplot(gs[row, col*9+1:col*9+7])
    ax.axhline(y=10,color="grey", linestyle="--", alpha=0.5)
    ax.axhline(y=20,color="grey", linestyle="--", alpha=0.5)
    ax.axhline(y=30,color="grey", linestyle="--", alpha=0.5)
    if durable == durables[0]:
        labels = ['Literature data']
    else:
        labels = ['_nolegend_']
    df_lt_d = df_lt[df_lt['durable'] == durable]
    sns.scatterplot(data=df_lt_d, x='reference year', y='mean', label=labels[0],color='black',zorder=no_of_runs2[durable], legend=False)
    for r in range(no_of_runs2[durable]):
        if durable == durables[0] and r == 0:
            labels = ['Variable lifetime model results']
        else:
            labels = ['_nolegend_']
        ax.plot(x, Dyn_MFA_System2.ExtraDict['mu'].Values[:,d,r], color='C1', alpha=0.1, label=labels[0])
    for r in range(no_of_runs1[durable]):
        if durable == durables[0] and r == 0:
            labels = ['Constant lifetime model results']
        else:
            labels = ['_nolegend_']
        ax.plot(x, Dyn_MFA_System1.ExtraDict['mu'].Values[:,d,r], color='C0', alpha=0.1, label=labels[0])
    ax.set_xlabel('Year')
    ax.set_ylabel('')
    ax.set_yticks([],labels=None)
    ax.set_title(durable.capitalize())
    ax.set_ylim(0, 38)
    ax.set_xlim(TimeStart, TimeEnd)
    if col == columns-1:
        col = 0
        row+= 1
    else:
        col += 1
lines_labels = [ax.get_legend_handles_labels() for ax in fig.axes]
lines, labels = [sum(lol, []) for lol in zip(*lines_labels)]
leg = fig.legend(reversed(lines), reversed(labels), bbox_to_anchor=(0.5,0.07), loc="upper center", fontsize=12)
for l in leg.get_lines(): # full answer: https://stackoverflow.com/questions/35200094/change-size-alpha-of-markers-in-the-legend-box
    l.set_alpha(1)
    # l.set_linewidth(5)
plt.show()
if export_figs_to_pdf:
    fig.savefig('outputs/Fig4.pdf', format='pdf', bbox_inches = "tight")

In [ ]:
if export_data_to_xlsx:
    parameter = Dyn_MFA_System1.ExtraDict['mu']
    array = get_array_with_nans(parameter.Values, parameter.Indices, no_of_runs1)[0,:-1,:]
    data, col_names = calculate_mean_and_percentiles(array,1)
    df_fig4constant = pd.DataFrame(data=data.T, index=pd.MultiIndex.from_product([durables], names=['durable']), columns=col_names)
    df_fig4constant.to_excel('outputs/Fig4_constant.xlsx')

In [ ]:
if export_data_to_xlsx:
    parameter = Dyn_MFA_System2.ExtraDict['mu']
    array = get_array_with_nans(parameter.Values, parameter.Indices, no_of_runs2)[:,:-1,:]
    data, col_names = calculate_mean_and_percentiles(array,2)
    df_fig4variable = pd.DataFrame(data=data.T, index=pd.MultiIndex.from_product([durables, MyYears], names=['durable','time']), columns=col_names)
    df_fig4variable.to_excel('outputs/Fig4_variable.xlsx')

## Calculations for the manuscript

### median and IQR values

In [ ]:
for durable in ['fridges', 'dishwashers']:
    d = Dyn_MFA_System1.IndexTable['Classification']['Durable'].Items.index(durable)
    print(f"{durable}, constant lifetime median {round(df_fig4constant['50th percentile'].loc[durable,],1)} with IQR of {round(df_fig4constant['25th percentile'].loc[durable,],1)} to {round(df_fig4constant['75th percentile'].loc[durable,],1)}")
for durable in ['fridges', 'dishwashers']:
    d = Dyn_MFA_System1.IndexTable['Classification']['Durable'].Items.index(durable)
    print(f"{durable}, variable lifetime start: median {round(df_fig4variable['50th percentile'].loc[durable,1940],1)} with IQR of {round(df_fig4variable['25th percentile'].loc[durable,1940],1)} to {round(df_fig4variable['75th percentile'].loc[durable,1940],1)}")
    print(f"{durable}, variable lifetime start: median {round(df_fig4variable['50th percentile'].loc[durable,2022],1)} with IQR of {round(df_fig4variable['25th percentile'].loc[durable,2022],1)} to {round(df_fig4variable['75th percentile'].loc[durable,2022],1)}")

### overlap of 5-95th percentile intervals

In [ ]:
def calculate_overlap(durable, year, lower='5th percentile', upper='95th percentile'):
    x1 = df_fig4constant[lower].loc[durable,]
    x2 = df_fig4constant[upper].loc[durable,]
    y1 = df_fig4variable[lower].loc[durable,year]
    y2 = df_fig4variable[upper].loc[durable,year]
    return max(0,min(x2,y2)-max(x1,y1))

df = pd.DataFrame(columns=['1940','2022','1940 (%)','2022 (%)'], index=durables)
for durable in durables:
    df['1940'].loc[durable] = calculate_overlap(durable, 1940)
    df['2022'].loc[durable] = calculate_overlap(durable, 2022)
    df['1940 (%)'].loc[durable] = calculate_overlap(durable, 1940)/(df_fig4constant['95th percentile'].loc[durable,]-df_fig4constant['5th percentile'].loc[durable,])
    df['2022 (%)'].loc[durable] = calculate_overlap(durable, 2022)/(df_fig4constant['95th percentile'].loc[durable,]-df_fig4constant['5th percentile'].loc[durable,])
df

### median lifetime decrease

In [ ]:
for durable in ['washing machines', 'ovens']:
    d = Dyn_MFA_System1.IndexTable['Classification']['Durable'].Items.index(durable)
    val1 = df_fig4variable['50th percentile'].loc[durable,1940]
    val2 = df_fig4variable['50th percentile'].loc[durable,2022]
    print(f'{durable}, lifetime decrease from median {round(val1,1)} to {round(val2,1)} ({round(((val2-val1)/val1)*100)}%)')

### inflection point

In [ ]:
for durable in ['washing machines', 'ovens']:
    d = Dyn_MFA_System1.IndexTable['Classification']['Durable'].Items.index(durable)
    no_of_runs = no_of_runs2[durable]
    perc_5th = np.nanpercentile(Dyn_MFA_System2.ExtraDict['LT-l-ti'].Values[0,d,:no_of_runs], 5)
    perc_95th = np.nanpercentile(Dyn_MFA_System2.ExtraDict['LT-l-ti'].Values[0,d,:no_of_runs], 95)
    print(f'{durable}, inflection point: 5th percentile {round(perc_5th)}, 95th percentile {round(perc_95th)}')

## SI figures

### Section 1

#### Inflows of appliances - various sources

In [ ]:
columns = 2
rows = 3
fig = plt.figure(figsize=(5*columns, 3*rows), constrained_layout=True)
# fig.set_dpi(dpi) #min 600 for publication
gs = fig.add_gridspec(rows, columns,hspace=0.1,wspace=0.05)
row = 0
col = 0
unique = df_i_data["source"].unique()
palette = dict(zip(unique, sns.color_palette("colorblind", n_colors=len(unique))))
markers = dict(zip(unique, ['o', 'v', '^', 's','P', 'X', 'D']))
for d,durable in enumerate(durables):
    df_i_d = df_i_data[df_i_data['durable'] == durable]
    ax1 = fig.add_subplot(gs[row, col])
    sns.scatterplot(ax=ax1, data=df_i_d, x='time', y='value', hue='source', style='source', hue_order= list(df_i_d['source'].unique()), markers=markers, palette=palette)
    ax1.legend('')
    ax1.get_legend().remove()
    ax1.set_xlabel('Year')
    ax1.set_ylabel('Sales/year')
    ax1.set_xlim(TimeStart-4, TimeEnd+4)
    ax1.set_title(durable.capitalize())
    if col == columns-1:
        col = 0
        row+= 1
    else:
        col += 1
lines_labels = [ax.get_legend_handles_labels() for ax in fig.axes]
lines, labels = [sum(lol, []) for lol in zip(*lines_labels)]
handle_list, label_list = [], []
for handle, label in zip(lines, labels):
    if label not in label_list:
        handle_list.append(handle)
        label_list.append(label)
fig.legend(handle_list, label_list, bbox_to_anchor=(0.5,0), loc="upper center")
plt.show()

In [ ]:
markers

In [ ]:
list(df_i_d['source'].unique())

#### POpD - various sources

In [ ]:
unique

In [ ]:
columns = 2
rows = 3
fig = plt.figure(figsize=(5*columns, 3*rows), constrained_layout=True)
# fig.set_dpi(dpi) #min 600 for publication
gs = fig.add_gridspec(rows, columns,hspace=0.1,wspace=0.05)
row = 0
col = 0
unique = df_popd["source"].unique()
markers = dict(zip(unique, ['o', 'v', '^', 's','P', 'D']))
palette = dict(zip(unique, sns.color_palette("colorblind", n_colors=len(unique))))
for d,durable in enumerate(durables):
    df_popd_d = df_popd[df_popd['durable'] == durable]
    df_popd_d = df_popd_d[df_popd_d['source'] != 'assumption']
    ax1 = fig.add_subplot(gs[row, col])
    sns.scatterplot(ax=ax1, data=df_popd_d, x='time', y='value', hue='source', style='source', hue_order= list(df_popd_d['source'].unique()), markers=markers, palette=palette)
    ax1.axhline(y=0.5,color="grey", linestyle="--", alpha=0.5)
    ax1.axhline(y=1,color="grey", linestyle="--", alpha=0.5)
    ax1.legend('')
    ax1.get_legend().remove()
    ax1.set_xlabel('Year')
    ax1.set_ylabel('Ownership')
    ax1.set_xlim(TimeStart-4, TimeEnd+4)
    ax1.set_ylim(-0.05,max(df_popd_d['value'])*1.2)
    ax1.set_title(durable.capitalize())
    if col == columns-1:
        col = 0
        row+= 1
    else:
        col += 1
lines_labels = [ax.get_legend_handles_labels() for ax in fig.axes]
lines, labels = [sum(lol, []) for lol in zip(*lines_labels)]
handle_list, label_list = [], []
for handle, label in zip(lines, labels):
    if label not in label_list:
        handle_list.append(handle)
        label_list.append(label)
fig.legend(handle_list, label_list, bbox_to_anchor=(0.5,0), loc="upper center")
plt.show()

#### Other parameters

In [ ]:
u_lt_k = 0.05   
u_p = 0.01     
u_ppd = 0.01    
u_lt_cab_l = 0.2 
u_lt_cab_k = 0.2 
u_soce = 0.2     
u_c = 0.01   

In [ ]:
columns = 2
rows = 2
fig = plt.figure(figsize=(5*columns, 3*rows), constrained_layout=True)
# fig.set_dpi(dpi) #min 600 for publication
gs = fig.add_gridspec(rows, columns,hspace=0.1,wspace=0.05)
# population
row=0
col=0
ax1 = fig.add_subplot(gs[row, col])
df = pd.read_excel(excel, sheet_name='P')
df_full = df
u = u_p
sns.scatterplot(ax=ax1,data=df, x='time', y='value', label='Data points', color='black', s=10,zorder=2)
sns.lineplot(ax=ax1,data=df_full, x='time', y='value', label='Full time series', color='gray',zorder=1)
ax1.fill_between(MyYears, df_full['value']*(1-u*1.96), df_full['value']*(1+u*1.96), color='gray', alpha=0.5, label='Uncertainty, ±1.96*StDev',zorder=0)
ax1.set_ylim(0,max(df['value'])*1.1)
ax1.set_xlim(TimeStart-4, TimeEnd+4)
ax1.set_xlabel('Year')
ax1.set_ylabel('Population')
ax1.set_title('Population')
ax1.legend('')
ax1.get_legend().remove()
# people per dwelling
row=0
col=1
ax1 = fig.add_subplot(gs[row, col])
df = pd.read_excel(excel, sheet_name='PpD_data')
df_full = pd.read_excel(excel, sheet_name='PpD')
u = u_ppd
sns.scatterplot(ax=ax1,data=df, x='time', y='value', label='Data points', color='black', s=10, zorder=2)
sns.lineplot(ax=ax1,data=df_full, x='time', y='value', label='Full time series', color='gray',zorder=1)
ax1.fill_between(MyYears, df_full['value']*(1-u*1.96), df_full['value']*(1+u*1.96), color='gray', alpha=0.5, label='Uncertainty, ±1.96*StDev',zorder=0)
ax1.set_ylim(0,max(df['value'])*1.1)
ax1.set_xlim(TimeStart-4, TimeEnd+4)
ax1.set_xlabel('Year')
ax1.set_ylabel('People per dwelling')
ax1.set_title('People per dwelling')
ax1.legend('')
ax1.get_legend().remove()
# stock of cabins
row=1
col=0
ax1 = fig.add_subplot(gs[row, col])
df = pd.read_excel(excel, sheet_name='C_data')
df_full = pd.read_excel(excel, sheet_name='C')
df_full = df_full[df_full['time'] >= 1940]
u = u_c
sns.scatterplot(ax=ax1,data=df, x='time', y='value', label='Data points', color='black', s=10,zorder=2)
sns.lineplot(ax=ax1,data=df_full, x='time', y='value', label='Full time series', color='gray',zorder=1)
ax1.fill_between(MyYears, df_full['value']*(1-u*1.96), df_full['value']*(1+u*1.96), color='gray', alpha=0.5, label='Uncertainty, ±1.96*StDev',zorder=0)
ax1.set_ylim(0,max(df['value'])*1.1)
ax1.set_xlim(TimeStart-4, TimeEnd+4)
ax1.set_xlabel('Year')
ax1.set_ylabel('Stock of cabins')
ax1.set_title('Stock of cabins')
ax1.legend('')
ax1.get_legend().remove()
# share of cabins electrified
row=1
col=1
ax1 = fig.add_subplot(gs[row, col])
df = pd.read_excel(excel, sheet_name='SoCE_data')
df = df[df['source'] != 'assumption']
df_assumption = pd.read_excel(excel, sheet_name='SoCE_data')
df_assumption = df_assumption[df_assumption['source'] == 'assumption']
df_full = pd.read_excel(excel, sheet_name='SoCE')
u = u_soce
sns.scatterplot(ax=ax1,data=df, x='time', y='value', label='Data points', color='black', s=20,zorder=2)
sns.scatterplot(ax=ax1,data=df_assumption, x='time', y='value', label='Assumption', color='gray',marker='D', s=20,zorder=2)
sns.lineplot(ax=ax1,data=df_full, x='time', y='value', label='Full time series', color='gray',zorder=1)
soce_max = df_full['value']*(1+u*1.96)
soce_max[soce_max>1] = 1
ax1.fill_between(MyYears, df_full['value']*(1-u*1.96), soce_max, color='gray', alpha=0.5, label='Uncertainty, ±1.96*StDev',zorder=0)
ax1.set_ylim(-0.05,1.1)
ax1.set_xlim(TimeStart-4, TimeEnd+4)
ax1.set_xlabel('Year')
ax1.set_ylabel('Share of cabins electrified')
ax1.set_title('Share of cabins electrified')
ax1.legend('')
ax1.get_legend().remove()
# legend
lines_labels = [ax.get_legend_handles_labels() for ax in fig.axes]
lines, labels = [sum(lol, []) for lol in zip(*lines_labels)]
handle_list, label_list = [], []
for handle, label in zip(lines, labels):
    if label not in label_list:
        handle_list.append(handle)
        label_list.append(label)
fig.legend(handle_list, label_list, bbox_to_anchor=(0.5,0), loc="upper center")
plt.show()

### Section 2

#### Sensitivity analysis

In [ ]:
sobol_St = {}
sobol_S1 = {}
sobol_St_CI = {}
sobol_S1_CI = {}
for d,durable in enumerate(Dyn_MFA_System3.IndexTable.Classification['Durable'].Items):
    problem_d = problem3
    sobol_indices = [SALib.analyze.sobol.analyze(problem3, Y) for Y in Dyn_MFA_System3.ExtraDict['LT-l'].Values[:,d,:no_of_runs3]]
    sobol_St[durable] = np.array([s['ST'] for s in sobol_indices])
    sobol_S1[durable] = np.array([s['S1'] for s in sobol_indices])
    sobol_St_CI[durable] = np.array([s['ST_conf'] for s in sobol_indices])
    sobol_S1_CI[durable] = np.array([s['S1_conf'] for s in sobol_indices])

In [ ]:
sobol_St_scaled = {durable: sobol_St[durable][-1, :]/np.sum(sobol_St[durable][-1, :]) for durable in Dyn_MFA_System3.IndexTable.Classification['Durable'].Items}

durables = Dyn_MFA_System3.IndexTable.Classification['Durable'].Items
weight_counts = {name: np.array([sobol_St_scaled[durable][i] for durable in Dyn_MFA_System3.IndexTable.Classification['Durable'].Items]) for i,name in enumerate(problem3['names'])}

fig, ax = plt.subplots()
bottom = np.zeros(len(durables))
for name, weight_count in weight_counts.items():
    p = ax.bar(durables, weight_count, 0.5, label=name, bottom=bottom)
    bottom += weight_count

# ax.set_title("Number of penguins with above average body mass")
handles, labels = ax.get_legend_handles_labels()
plt.xticks(rotation=90)
ax.legend(handles[::-1], labels[::-1], loc="upper left", bbox_to_anchor=(1, 1))

plt.show()

In [ ]:
if export_data_to_xlsx:
    df = pd.DataFrame.from_dict(data=weight_counts, orient='index', columns=durables)
    df.to_excel('outputs/FigS4.xlsx')

### Section 3

#### Number of dwellings

In [ ]:
df_ppd = pd.read_excel(excel, sheet_name='PpD')
df_p = pd.read_excel(excel, sheet_name='P')
df_c = pd.read_excel(excel, sheet_name='C')
df_soce = pd.read_excel(excel, sheet_name='SoCE')
df_k_cab = pd.read_excel(excel, sheet_name='k-cab')
df_l_cab = pd.read_excel(excel, sheet_name='lambda-cab')

In [ ]:
dwellings = np.array(df_p['value']/df_ppd['value'])

t = df_c['time']
s = df_c['value']
scale = df_l_cab['value'].values[0]
shape = df_k_cab['value'].values[0]
sf = np.zeros((len(t), len(t)))
for m in range(0, len(t)):  # cohort index
    sf[m::,m] = scipy.stats.weibull_min.sf(np.arange(0,len(t)-m), c=shape, loc = 0, scale=scale)

# MFA calculations start (assuming sf[0] != 0 and no negative inflows)
i = np.zeros(len(t))
s_c = np.zeros((len(t), len(t)))
i[0] = s[0] / sf[0, 0]
s_c[:, 0] = i[0] * sf[:, 0]
for m in range(1, len(t)):
    i[m] = (s[m] - s_c[m, :].sum()) / sf[m,m]
    s_c[m::, m] = i[m] * sf[m::, m]

o_c = np.zeros_like(s_c)
o_c[1::,:] = -1 * np.diff(s_c,n=1,axis=0)
o_c[np.diag_indices(len(t))] = i - np.diag(s_c) # allow for outflow in year 0 already

soce = df_soce['value'] # share of cabins electrified
soce[soce >1] = 1
soce[:1960-TimeStart] = 0 
soce[soce <0] = 0
el_cabins = np.einsum('tc,c->t',s_c[40:,40:],soce)
all_dwellings = dwellings+el_cabins

In [ ]:
fig = plt.figure(figsize=(5, 3), constrained_layout=True)
ax = fig.add_subplot()
ax.fill_between(MyYears, np.zeros_like(dwellings), dwellings/10**6, color='gray', label='Primary dwellings', alpha=0.5)
ax.fill_between(MyYears, dwellings/10**6, all_dwellings/10**6, color='gray', label='Recreational cabins',alpha=1)
a = (all_dwellings[-1] - all_dwellings[0])/(MyYears[-1] - MyYears[0])
b = all_dwellings[0] - a*MyYears[0]
ax.plot(MyYears, (a*np.array(MyYears)+b)/10**6)
ax.set_ylabel('Dwelling stock (million)')
ax.set_xlabel('Year')
plt.legend(loc='upper left')

In [ ]:
print(f'all dwellings in 1940: {round(all_dwellings[0])} ({round(el_cabins[0]/all_dwellings[0]*100)}% cabins) and in  2022: {round(all_dwellings[-1])} ({round(el_cabins[-1]/all_dwellings[-1]*100)}% cabins)')
perc_inc_p_y = np.concatenate([np.array([0]),np.diff(all_dwellings)])/all_dwellings
print(f'the average increase per year in the total number of dwellings was {np.round(np.mean(perc_inc_p_y)*100,2)}%')

In [ ]:
if export_data_to_xlsx:
    data = np.concatenate(([dwellings],[el_cabins]), axis=0)
    df = pd.DataFrame(data=data.T, index=pd.MultiIndex.from_product([MyYears], names=['time']), columns=['primary dwellings', 'secondary dwellings'])
    df.to_excel('outputs/FigS5.xlsx')

#### Age of appliances

In [ ]:
columns = 2
rows = 3
fig = plt.figure(figsize=(6*columns, 3.5*rows), constrained_layout=True)
# fig.set_dpi(dpi) #min 600 for publication
gs = fig.add_gridspec(rows, columns,hspace=0.1)
row = 0
col = 0
for d,durable in enumerate(durables):
    ax = fig.add_subplot(gs[row, col])
    for r in range(no_of_runs1[durable]):
        ax.plot(MyYears, Dyn_MFA_System1.ExtraDict['age_o'].Values[:,d,r], color='C0', alpha=0.1, label='Constant lifetime model results', zorder=r)
        # ax.plot(MyYears, age_o[:,d,r], color='C1', alpha=0.1, zorder=r)
    for r in range(no_of_runs2[durable]):
        ax.plot(MyYears, Dyn_MFA_System2.ExtraDict['age_o'].Values[:,d,r], color='C1', alpha=0.1, label='Variable lifetime model results', zorder=r)
    ax.set_xlabel('Year')
    ax.set_ylabel('Age of outflows (years)')
    ax.set_title(durable.capitalize())
    # add validation data
    df_lt_d = df_lt[df_lt['durable'] == durable]
    xdata = df_lt_d['reference year'].values
    ydata = df_lt_d['mean'].values
    ax.scatter(xdata,ydata,color='black', label='Literature data',zorder=r+1)
    ax.set_xlim(1940-4,2022+4)
    ax.set_ylim(0,28)  
    if col == columns-1:
        col = 0
        row+= 1
    else:
        col += 1
lines_labels = [ax.get_legend_handles_labels() for ax in fig.axes]
lines, labels = [sum(lol, []) for lol in zip(*lines_labels)]
handle_list, label_list = [], []
for handle, label in zip(lines, labels):
    if label not in label_list:
        handle_list.append(handle)
        label_list.append(label)
leg = fig.legend(handle_list, label_list, bbox_to_anchor=(0.5,0), loc="upper center", fontsize=12)
for l in leg.get_lines(): # full answer: https://stackoverflow.com/questions/35200094/change-size-alpha-of-markers-in-the-legend-box
    l.set_alpha(1)
plt.show()

In [ ]:
if export_data_to_xlsx:
    parameter = Dyn_MFA_System1.ExtraDict['age_o']
    array = get_array_with_nans(parameter.Values, parameter.Indices, no_of_runs1)
    data, col_names = calculate_mean_and_percentiles(array,2)
    df = pd.DataFrame(data=data.T, index=pd.MultiIndex.from_product([durables, MyYears], names=['durable','time']), columns=col_names)
    df.to_excel('outputs/FigS6_constant.xlsx')

In [ ]:
if export_data_to_xlsx:
    parameter = Dyn_MFA_System2.ExtraDict['age_o']
    array = get_array_with_nans(parameter.Values, parameter.Indices, no_of_runs2)
    data, col_names = calculate_mean_and_percentiles(array,2)
    df = pd.DataFrame(data=data.T, index=pd.MultiIndex.from_product([durables, MyYears], names=['durable','time']), columns=col_names)
    df.to_excel('outputs/FigS6_variable.xlsx')

In [ ]:
columns = 2
rows = 3
fig = plt.figure(figsize=(6*columns, 3.5*rows), constrained_layout=True)
# fig.set_dpi(dpi) #min 600 for publication
gs = fig.add_gridspec(rows, columns,hspace=0.1)
row = 0
col = 0
for d,durable in enumerate(durables):
    ax = fig.add_subplot(gs[row, col])
    for r in range(no_of_runs1[durable]):
        ax.plot(MyYears, Dyn_MFA_System1.ExtraDict['age_o_i'].Values[:,d,r], color='C0', alpha=0.1, label='Constant lifetime model results', zorder=r)
        # ax.plot(MyYears, age_o[:,d,r], color='C1', alpha=0.1, zorder=r)
    for r in range(no_of_runs2[durable]):
        ax.plot(MyYears, Dyn_MFA_System2.ExtraDict['age_o_i'].Values[:,d,r], color='C1', alpha=0.1, label='Variable lifetime model results', zorder=r)
    ax.set_xlabel('Year')
    ax.set_ylabel('Age of outflows (years)')
    ax.set_title(durable.capitalize())
    # add validation data
    df_lt_d = df_lt[df_lt['durable'] == durable]
    xdata = df_lt_d['reference year'].values
    ydata = df_lt_d['mean'].values
    ax.scatter(xdata,ydata,color='black', label='Literature data',zorder=r+1)
    ax.set_xlim(1940-4,2022+4)
    ax.set_ylim(0,28)  
    if col == columns-1:
        col = 0
        row+= 1
    else:
        col += 1
lines_labels = [ax.get_legend_handles_labels() for ax in fig.axes]
lines, labels = [sum(lol, []) for lol in zip(*lines_labels)]
handle_list, label_list = [], []
for handle, label in zip(lines, labels):
    if label not in label_list:
        handle_list.append(handle)
        label_list.append(label)
leg = fig.legend(handle_list, label_list, bbox_to_anchor=(0.5,0), loc="upper center", fontsize=12)
for l in leg.get_lines(): # full answer: https://stackoverflow.com/questions/35200094/change-size-alpha-of-markers-in-the-legend-box
    l.set_alpha(1)
plt.show()

In [ ]:
if export_data_to_xlsx:
    parameter = Dyn_MFA_System1.ExtraDict['age_o_i']
    array = get_array_with_nans(parameter.Values, parameter.Indices, no_of_runs1)
    data, col_names = calculate_mean_and_percentiles(array,2)
    df = pd.DataFrame(data=data.T, index=pd.MultiIndex.from_product([durables, MyYears], names=['durable','time']), columns=col_names)
    df.to_excel('outputs/FigS7_constant.xlsx')

In [ ]:
if export_data_to_xlsx:
    parameter = Dyn_MFA_System2.ExtraDict['age_o_i']
    array = get_array_with_nans(parameter.Values, parameter.Indices, no_of_runs2)
    data, col_names = calculate_mean_and_percentiles(array,2)
    df = pd.DataFrame(data=data.T, index=pd.MultiIndex.from_product([durables, MyYears], names=['durable','time']), columns=col_names)
    df.to_excel('outputs/FigS7_variable.xlsx')

## Plot and export the scale parameter

In [ ]:
from scipy import stats
mean = {k: np.zeros((len(durables))) for k in ['ti', 'a', 'C0', 'C1', 'constant']}
mode = {k: np.zeros((len(durables))) for k in ['ti', 'a', 'C0', 'C1', 'constant']}
median = {k: np.zeros((len(durables))) for k in ['ti', 'a', 'C0', 'C1', 'constant']}
for d,durable in enumerate(durables):
    mode['constant'][d] = stats.mode(np.round(Dyn_MFA_System1.ExtraDict['LT-l'].Values[0,d,:no_of_runs2[durable]],0)).mode
    mode['ti'][d] = stats.mode(np.round(Dyn_MFA_System2.ExtraDict['LT-l-ti'].Values[0,d,:no_of_runs2[durable]],0)).mode
    mode['a'][d] = stats.mode(np.round(Dyn_MFA_System2.ExtraDict['LT-l-a'].Values[0,d,:no_of_runs2[durable]],1)).mode
    mode['C0'][d] = stats.mode(np.round(Dyn_MFA_System2.ExtraDict['LT-l-C0'].Values[0,d,:no_of_runs2[durable]],1)).mode
    mode['C1'][d] = stats.mode(np.round(Dyn_MFA_System2.ExtraDict['LT-l-C1'].Values[0,d,:no_of_runs2[durable]],1)).mode
    mean['constant'][d] = np.mean(Dyn_MFA_System1.ExtraDict['LT-l'].Values[0,d,:no_of_runs2[durable]])
    mean['ti'][d] = np.mean(Dyn_MFA_System2.ExtraDict['LT-l-ti'].Values[0,d,:no_of_runs2[durable]])
    mean['a'][d] = np.mean(Dyn_MFA_System2.ExtraDict['LT-l-a'].Values[0,d,:no_of_runs2[durable]])
    mean['C0'][d] = np.mean(Dyn_MFA_System2.ExtraDict['LT-l-C0'].Values[0,d,:no_of_runs2[durable]])
    mean['C1'][d] = np.mean(Dyn_MFA_System2.ExtraDict['LT-l-C1'].Values[0,d,:no_of_runs2[durable]])
    median['constant'][d] = np.median(Dyn_MFA_System1.ExtraDict['LT-l'].Values[0,d,:no_of_runs2[durable]])
    median['ti'][d] = np.median(Dyn_MFA_System2.ExtraDict['LT-l-ti'].Values[0,d,:no_of_runs2[durable]])
    median['a'][d] = np.median(Dyn_MFA_System2.ExtraDict['LT-l-a'].Values[0,d,:no_of_runs2[durable]])
    median['C0'][d] = np.median(Dyn_MFA_System2.ExtraDict['LT-l-C0'].Values[0,d,:no_of_runs2[durable]])
    median['C1'][d] = np.median(Dyn_MFA_System2.ExtraDict['LT-l-C1'].Values[0,d,:no_of_runs2[durable]])

In [ ]:
def logistic(x, ti, a, C0, C1):
    return (C1 - C0) / (1 + np.exp(-a * (x - ti))) + C0

In [ ]:
x = Dyn_MFA_System1.IndexTable.Classification['Time'].Items
columns = 2
rows = 3
fig = plt.figure(figsize=(6*columns, 3.5*rows), constrained_layout=True)
gs = fig.add_gridspec(rows, columns,hspace=0.1)
row = 0
col = 0
for d,durable in enumerate(Dyn_MFA_System1.IndexTable['Classification']['Durable'].Items):
    ax = fig.add_subplot(gs[row, col])
    if durable == durables[0]:
        labels = ['constant mean', 'constant mode', 'constant median', 'logistic mean','logistic mode', 'logistic median']
    else:
        labels = ['_nolegend_']*6
    ax.plot(x, mean['constant'][d]*np.ones_like(x), label=labels[0], color='C0', linestyle='-')
    ax.plot(x, mode['constant'][d]*np.ones_like(x), label=labels[1], color='C0', linestyle='--')
    ax.plot(x, median['constant'][d]*np.ones_like(x), label=labels[2], color='C0', linestyle=':')
    ax.plot(x, logistic(MyYears, mean['ti'][d], mean['a'][d], mean['C0'][d], mean['C1'][d]), label=labels[3], color='C1', linestyle='-')
    ax.plot(x, logistic(MyYears, mode['ti'][d], mode['a'][d], mode['C0'][d], mode['C1'][d]), label=labels[4], color='C1', linestyle='--')
    ax.plot(x, logistic(MyYears, median['ti'][d], median['a'][d], median['C0'][d], median['C1'][d]), label=labels[5], color='C1', linestyle=':')
    ax.set_xlabel('Year')
    ax.set_ylabel('Appliance ownership')
    ax.set_title(durable.capitalize())
    ax.set_xlim(1940-4,2022+4)
    ax.set_ylim(0,30)
    ax.legend('')
    if col == columns-1:
        col = 0
        row+= 1
    else:
        col += 1
lines_labels = [ax.get_legend_handles_labels() for ax in fig.axes]
lines, labels = [sum(lol, []) for lol in zip(*lines_labels)]
leg = fig.legend(lines, labels, bbox_to_anchor=(0.5,0), loc="upper center", fontsize=12)
plt.show()

In [ ]:
df = pd.DataFrame(data=mean['constant'].flatten(order='F'), index=pd.MultiIndex.from_product([durables], names=['durable']), columns=['value'])
df.to_excel('outputs/scale_mean_constant.xlsx')

data = np.zeros((len(MyYears),len(durables)))
for d, durable in enumerate(durables):
    data[:,d] = logistic(MyYears, mean['ti'][d], mean['a'][d], mean['C0'][d], mean['C1'][d])
df = pd.DataFrame(data=data.flatten(order='F'), index=pd.MultiIndex.from_product([durables,MyYears], names=['durable','time']), columns=['value'])
df.to_excel('outputs/scale_mean_variable.xlsx')